In [ ]:
mod = SourceModule("""
    #include <stdio.h>
    __global__ main_gpu(, ushort *component, ushort num_components, ushort *color, ushort num_vtds, ushort *nbrs, ushort *nbr_rng)
    {
        for(w = 0; w < num_vtds; w++)
    }
    
    
    __device__ void compute_components_local(ushort w, ushort *component, ushort num_components, ushort *color, ushort num_vtds, ushort *nbrs, ushort *nbr_rng)
    {    
        const uint blockId = blockIdx.z*(gridDim.y*gridDim.x) + blockIdx.y*(gridDim.x) + blockIdx.x;
        const uint threadIdLoc = threadIdx.z*(blockDim.y*blockDim.x) + threadIdx.y*(blockDim.x) + threadIdx.x;
        const uint threadIdGlob = blockId*(blockDim.z*blockDim.y*blockDim.x) + threadIdLoc;

        const char *div1 = "***********************************************************************************************\\n";
        const char *div2 = "###############################################################################################\\n";

        /*
        printf("%sHello from block (%d,%d,%d) and thread (%d,%d,%d)\\n%s",
                div1, blockIdx.x, blockIdx.y, blockIdx.x, threadIdx.x, threadIdx.y, threadIdx.x, div2);
        */

        
        int u, v, w;
        num_components = 0;
        //printf("%d\\n",num_vtds);
        //printf("%d\\n",num_components);
        for(w = 0; w < num_vtds; w++)
        {
            printf("\\n\\nLooking at vtd %d\\n\\n",w);
            component[w] = num_components;
            //printf("vtd w=%d is in component %d\\n", w,component[w]);
            for(i = nbr_rng[w]; i = nbr_rng[w+1]; i++)
            {
                v = nbrs[i];
                if(color[v] == color[w])
                {
                    printf("Matched color on %d and %d.\\n", v, w);
                    printf("vtd v=%d is in component %d\\n", v,component[v]);
                    if(component[v] < component[w])
                    {
                        printf("%d and %d were adjacent, with comp[%d] < comp[%d].\\n", v, w, v, w);
                        component[w] = component[v];
                    }
                }
            }
            
            if(component[w] == num_components)
            {
                //printf("Incrementing num_components to %d",num_components);
                num_components++;
            }
                        
            for(i = nbr_rng[w]; i = nbr_rng[w+1]; i++)
            {
                v = nbrs[i]
                if(color[v] == color[w])
                {
                    if(component[v] > component[w])
                    {
                        old_comp = component[v];
                        new_comp = component[w];
                        for(u = 0; u < w; u++)
                        {
                            if(component[u] == old_component)
                            {
                                printf("Updating component of %d from %d to %d\\n", u, component[u], new_comp);
                                component[u] = new_comp;
                            }
                        }
                    }
                }
            }
        }
        printf("MADE IT\\n");
    }
    
    __global__ void compute_components_global(ushort *component, ushort num_components, ushort *color, ushort num_vtds, ushort *nbrs, ushort *nbr_rng)
    {

""")


In [1]:
from helper.setup import *
from helper.pycuda_setup import *
df = pd.DataFrame()
df['A'] = [1,2,3]
df['B'] = [7,2,3]
display(df)
df_gpu = togpu(df,'uint16')
print(df_gpu)

,A,B
0,1,7
1,2,2
2,3,3


[1 7 2 2 3 3]


In [14]:
from helper.setup import *
nbrs = [[1],[0,2],[1]]
deg = np.array([len(n) for n in nbrs]).astype('uint8')
nbr_nrg = np.insert(deg.cumsum(),0,0).astype('uint16')
print(np.asarray(nbrs))
print(np.concatenate(nbrs))
N
#display(nbr_nrg)
#help(np.insert)
#N = np.asarray(nbrs)
#print(N.flatten())
# idx = 0
# idx_ = 
# display(neighbors)
# display(ind_end)

[[1] [0, 2] [1]]
[1 0 2 1]


array([1, 0, 2, 1])

In [34]:
num_vtd = 10
num_color = 3
num_edges = 20
adj = np.random.randint(2,size=[num_vtd,num_vtd]).astype('uint8')
adj = np.tril(adj,k=-1)
adj = adj + adj.T
np.alltrue(A==A.T)
print(adj.dtype)
color = np.random.randint(num_color, size = num_vtd).astype('uint16')
component = np.arange(num_vtd).astype('uint16')
vtd0 = np.zeros(num_edges).astype('uint32')
vtd1 = np.zeros(num_edges).astype('uint32')



uint8


In [2]:
from helper.setup import *
from helper.pycuda_setup import *

mod = SourceModule("""
    #include <stdio.h>
    __global__ void compute_components_gpu(char *adjacent, char *color)
    {
        const uint blockId = blockIdx.z*(gridDim.y*gridDim.x) + blockIdx.y*(gridDim.x) + blockIdx.x;
        const uint threadIdLoc = threadIdx.z*(blockDim.y*blockDim.x) + threadIdx.y*(blockDim.x) + threadIdx.x;
        const uint threadIdGlob = blockId*(blockDim.z*blockDim.y*blockDim.x) + threadIdLoc;

        const char *div1 = "***********************************************************************************************\\n";
        const char *div2 = "###############################################################################################\\n";
    
        printf("%s hello %s",div1, div2);
    }    
""")
compute_components = mod.get_function("compute_components_gpu")

/home/particle/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:17: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(8): warning: variable "threadIdGlob" was declared but never referenced

kernel.cu(8): warning: variable "threadIdGlob" was declared but never referenced


